In [7]:
import requests
import json
import pprint
import math

from pyecharts.charts import BMap
from pyecharts import options as opts
from pyecharts.globals import BMapType, ChartType

In [8]:
url = 'http://map.amap.com/service/subway?_1615466846985&srhdata=1100_drw_beijing.json'
response = requests.get(url)
result = json.loads(response.text)
stations = []
for i in result['l']:
    station = []
    for a in i['st']:
        station.append([float(b) for b in a['sl'].split(',')])
    stations.append(station)
pprint.pprint(stations)

[[[116.178945, 39.925686],
  [116.162586, 39.923298],
  [116.13401, 39.91603],
  [116.125809, 39.912383],
  [116.122225, 39.905138],
  [116.123254, 39.89578],
  [116.114443, 39.890465],
  [116.100334, 39.889378]],
 [[116.190337, 39.90745],
  [116.212684, 39.907442],
  [116.235948, 39.90744],
  [116.252888, 39.907433],
  [116.273987, 39.907456],
  [116.294765, 39.907474],
  [116.309919, 39.907469],
  [116.321459, 39.907422],
  [116.337583, 39.907379],
  [116.352583, 39.907234],
  [116.356866, 39.907242],
  [116.374072, 39.907383],
  [116.391278, 39.907472],
  [116.401216, 39.90778],
  [116.411571, 39.908069],
  [116.41848, 39.908325],
  [116.435806, 39.908501],
  [116.450497, 39.908454],
  [116.461841, 39.909104],
  [116.475783, 39.908287],
  [116.495456, 39.908749],
  [116.515664, 39.908495],
  [116.531421, 39.909448],
  [116.554639, 39.909215],
  [116.57676, 39.91014],
  [116.599002, 39.90909],
  [116.618658, 39.906121],
  [116.637252, 39.903864],
  [116.646621, 39.893329],
  [116.657

## 转换地理坐标位置

In [9]:
pi = 3.1415926535897932384 #π
r_pi = pi * 3000.0/180.0

def gcj02_bd09(lon_gcj02,lat_gcj02):
    b = math.sqrt(lon_gcj02 * lon_gcj02 + lat_gcj02 * lat_gcj02) + 0.00002 * math.sin(lat_gcj02 * r_pi)
    o = math.atan2(lat_gcj02 , lon_gcj02) + 0.000003 * math.cos(lon_gcj02 * r_pi)
    lon_bd09 = b * math.cos(o) + 0.0065
    lat_bd09 = b * math.sin(o) + 0.006
    return [lon_bd09,lat_bd09]

In [10]:
# 将GCJ-02坐标系转成BD-09的函数，调用这个函数，就可以将高德地图获取的坐标点集合统统转换成百度坐标系
result = []
for station in stations:
    result.append([gcj02_bd09(*point) for point in station])
result

[[[116.18536104177846, 39.931870332777684],
  [116.1689495644047, 39.929637193502124],
  [116.14047096593411, 39.92209777735635],
  [116.13232364276523, 39.91830323830973],
  [116.12876737351012, 39.91099833790209],
  [116.12979954771687, 39.90166091594628],
  [116.12103910763038, 39.896214717144446],
  [116.10696363506538, 39.89503318291807]],
 [[116.19683191959666, 39.91344137919388],
  [116.21928785916374, 39.913116162344174],
  [116.24252116760917, 39.913203497663204],
  [116.25936278739741, 39.91348307293766],
  [116.28036596313514, 39.91378520497797],
  [116.30117209248412, 39.91372123905778],
  [116.31641128123103, 39.91346800566625],
  [116.32801943127427, 39.913222503000036],
  [116.34419463547177, 39.91303033287667],
  [116.35917066810045, 39.91295560048145],
  [116.36343499110994, 39.91301802294884],
  [116.38053953444282, 39.9134544275225],
  [116.39766333200939, 39.913782916820196],
  [116.40758902328636, 39.914125926092105],
  [116.41796482707578, 39.91435341656478],
  [1

## 地理可视化

In [12]:
map_b = (
    BMap(init_opts = opts.InitOpts(width = "800px", height = "600px"))
    .add_schema(
        baidu_ak = 'OfpmmmIKaWP427E5mN6Whe9t48CwbyMN', #百度地图开发应用appkey
        center = [116.403963, 39.915119], #当前视角的中心点
        zoom = 10, #当前视角的缩放比例
        is_roam = True, #开启鼠标缩放和平移漫游
    )
    .add(
        series_name = "",
        type_ = ChartType.LINES, #设置Geo图类型
        data_pair = result, #数据项
        is_polyline = True, #是否是多段线，在画lines图情况下#
        linestyle_opts = opts.LineStyleOpts(color = "blue", opacity = 0.5, width = 1), # 线样式配置项
    )
    .add_control_panel(
        maptype_control_opts = opts.BMapTypeControlOpts(type_ = BMapType.MAPTYPE_CONTROL_DROPDOWN), #切换地图类型的控件
        scale_control_opts = opts.BMapScaleControlOpts(), #比例尺控件
        overview_map_opts = opts.BMapOverviewMapControlOpts(is_open = True), #添加缩略地图
        navigation_control_opts = opts.BMapNavigationControlOpts() #地图的平移缩放控件
    )
)

map_b.render(path = 'subway_beijing.html')

'E:\\Desktop\\GIT\\TMP\\subway\\subway_beijing.html'